In [12]:
#実行場所
import os
os.chdir(r"C:\Users\tsuchida\Documents\techgym_セミナー\TortoiseGit_resorce\techgym_ai\Chapter_1\Answer_sheet\AI_Chapter1_saved_files")

In [14]:
#AI-TECHGYM-1-13-A-3
#自然言語処理

#word2vec データ読み込み
from gensim.models import KeyedVectors
import pandas as pd
import urllib.request

import os
os.chdir(r"C:\Users\tsuchida\Documents\techgym_セミナー\TortoiseGit_resorce\techgym_ai\Chapter_1\Answer_sheet\AI_Chapter1_saved_files")

title = "stanby-jobs-200d-word2vector.bin"
if not os.path.exists(title):
    print(title + " DOWNLOAD.")
    url = "https://github.com/bizreach/ai/releases/download/2018-03-13/stanby-jobs-200d-word2vector.bin"
    urllib.request.urlretrieve(url,"{0}".format(title))
else :
    print(title + " EXIST.")

# ダウンロード先のパスを指定
MODEL_FILENAME = "./stanby-jobs-200d-word2vector.bin"
w2v = KeyedVectors.load_word2vec_format(MODEL_FILENAME, binary=True)

# データの読み込み
df = pd.read_csv("word.csv") 

# 読み込んだデータのサンプルを取得
co_df = df
display(co_df.head())

# 閾値を決める
THRESHOLD = 0.7

# 結果をlistで保存
result = []

# 類似度を計算
for i in df["words"].values:
    for j in co_df["words"].values:
        tmp = []
        # word2vecにない単語はスルーする
        try:
            similarity = w2v.similarity(i, j)
            # 類似度が0.99以上は同じ単語とする
            # 類似度がTHRESHOLD以上のものを抽出
            if similarity < 0.99 and similarity > THRESHOLD:
                tmp = [i,j,str(similarity)]
                result.append(tmp)
        except:
            pass

#リストを表示
print(result)

#データフレーム
df_result = pd.DataFrame(result,columns=["単語A","単語B","類似度"])
display(df_result)

#CSVファイルに保存
df_result.to_csv('./w2v_result.csv',index=False)



stanby-jobs-200d-word2vector.bin EXIST.


,words
0,合計
1,遣唐使
2,熱帯魚
3,向け
4,焼き


[['関東', '関西', '0.7552338670281852'], ['北', '西', '0.818989671306523'], ['海', '山', '0.7102969980315794'], ['長崎', '福岡', '0.7406691703356508'], ['福岡', '長崎', '0.7406691703356508'], ['福岡', '東京', '0.7367493216623424'], ['東京', '福岡', '0.7367493216623424'], ['いろいろ', '色々', '0.7147815817895989'], ['関西', '関東', '0.7552338670281852'], ['関西', '九州', '0.7193674742629778'], ['九州', '関西', '0.7193674742629778'], ['山', '海', '0.7102969980315794'], ['西', '北', '0.818989671306523'], ['色々', 'いろいろ', '0.7147815817895989']]


,単語A,単語B,類似度
0,関東,関西,0.7552338670281852
1,北,西,0.818989671306523
2,海,山,0.7102969980315794
3,長崎,福岡,0.7406691703356508
4,福岡,長崎,0.7406691703356508
5,福岡,東京,0.7367493216623424
6,東京,福岡,0.7367493216623424
7,いろいろ,色々,0.7147815817895989
8,関西,関東,0.7552338670281852
9,関西,九州,0.7193674742629778


In [5]:
w2v.n_similarity('テクノロジー','金融')

0.3156337554692984

In [10]:
'金融' in w2v.index2entity

True

In [3]:
import itertools

lis = [1,2,3,4]
for pair in itertools.combinations(lis, 2):
    print(pair[0])
    print(type(pair))

1
<class 'tuple'>
1
<class 'tuple'>
1
<class 'tuple'>
2
<class 'tuple'>
2
<class 'tuple'>
3
<class 'tuple'>


In [ ]:
display(set(trans[trans.StockCode=='85123A'].Description))

In [ ]:
display(set(trans[trans.StockCode=='47566'].Description))

In [ ]:
display(trans[trans.StockCode=='85123A'].Description.value_counts())

In [ ]:
display(trans.head())

In [ ]:
import numpy as np

df_view = pd.DataFrame(pd.date_range('2000-01-02', freq='1D', periods=15), columns=['Date'])

bins_dt = pd.date_range('2000-01-01', freq='3D', periods=6)
bins_str = bins_dt.astype(str).values

labels = ['({}, {}]'.format(bins_str[i-1], bins_str[i]) for i in range(1, len(bins_str))]

df['cat'] = pd.cut(df.Date.astype(np.int64)//10**9, bins=bins_dt.astype(np.int64)//10**9, labels=labels)

In [ ]:
df_view = pd.concat([trans[trans.StockCode=='85123A'], trans[trans.StockCode=='47566']], axis=0)
display(df_view.head(1))

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
df_view = df_view.set_index('InvoiceDate')
df_view.plot.bar()